In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available 14 qubit machine instead
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_20_tokyo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_almaden') from IBMQ(hub='ibm-q-ncsu', group='nc-state

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the 20qubit boeblingen instead
device = provider.get_backend('ibmq_boeblingen')
backend_monitor(device)
device

ibmq_boeblingen
Configuration
-------------
    n_qubits: 20
    operational: True
    status_msg: active
    pending_jobs: 187
    backend_version: 1.0.3
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    max_experiments: 900
    coupling_map: [[0, 1], [1, 0], [1, 2], [1, 6], [2, 1], [2, 3], [3, 2], [3, 4], [3, 8], [4, 3], [5, 6], [5, 10], [6, 1], [6, 5], [6, 7], [7, 6], [7, 8], [7, 12], [8, 3], [8, 7], [8, 9], [9, 8], [9, 14], [10, 5], [10, 11], [11, 10], [11, 12], [11, 16], [12, 7], [12, 11], [12, 13], [13, 12], [13, 14], [13, 18], [14, 9], [14, 13], [15, 16], [16, 11], [16, 15], [16, 17], [17, 16], [17, 18], [18, 13], [18, 17], [18, 19], [19, 18]]
    allow_q_object: True
    sample_name: HexV2
    dt: 3.5555555555555554
    n_uchannels: 46
    qubit_lo_range: [[4.7469014458963175, 5.346901445896317], [4.547291237462082, 5.147291237462082], [4.4036557596521675, 5.003655759652167], [4.474323292627603, 5.074323292627603], [4.06619110415614, 4.66

<IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>

In [19]:
transQcList = []

pi = math.pi

In [13]:
import numpy as np
def input_state(circ, nqubits, value):
    for j in range(nqubits):
        circ.h(j)
        circ.u1(-value * math.pi/float(2**(j)), j)
def qft(circ, q, n):
    """n-qubit QFT on q in circ."""
    for j in range(n):
        for k in range(j):
            circ.cu1(np.pi / float(2**(j - k)), q[j], q[k])
        circ.h(q[j])

In [14]:
def qftExperimentClassical():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    
    for i in range(4):
        circuit.h(q[i])
    
    qft(circuit, q, 4)

    for k in range(4):
        circuit.measure(q[k],c[k])
    
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QFT without classical assertion: size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())

    
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    for i in range(4):
        circuit.h(q[i])

    qft(circuit, q, 4)

    qubitList = [q[3]]
    success = classical_assertion(circuit, qubitList, 0)
    if not success:
        print("Assertion circuit error")

    for k in range(4):
        circuit.measure(q[k],c[k])
        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QFT with classical assertion: size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())


In [15]:
def qftExperimentSuperposition():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    
    circuit.x(q[2])
    #swap 1 and 3
    for j in range(4):
        m = j
        if j == 1:
            m = 3
        if j == 3:
            m = 1
        for k in range(j):
            n = k
            if k == 1:
                n = 3
            if k == 3:
                n = 1
                
            circuit.cu1(np.pi / float(2**(j - k)), q[m], q[n])
        circuit.h(q[m])


    circuit.u1(-pi/4, q[0])
    circuit.h(q[0])
    circuit.u1(-pi/2, q[3])
    circuit.h(q[3])
    circuit.u1(-pi, q[2])
    circuit.h(q[2])
    circuit.h(q[1])
    circuit.measure(q[0],c[0])
    circuit.measure(q[3],c[1])
    circuit.measure(q[2],c[2])
    circuit.measure(q[1],c[3])
    
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QFT without superposition assertion size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())

    
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    circuit.x(q[2])
    #swap 1 and 3
    for j in range(4):
        m = j
        if j == 1:
            m = 3
        if j == 3:
            m = 1
        for k in range(j):
            n = k
            if k == 1:
                n = 3
            if k == 3:
                n = 1
                
            circuit.cu1(np.pi / float(2**(j - k)), q[m], q[n])
        circuit.h(q[m])

    circuit.u1(-pi/4, q[0])
    circuit.h(q[0])

    circuit.u1(-pi, q[2])
    circuit.h(q[2])
    circuit.h(q[1])
    
    #assert for q[3]
    qubitList = [q[3]]
    phaseDict = {q[3]:[pi/2, pi/2]}
    success = superposition_assertion(circuit, qubitList, phaseDict, flag = 1)
    if not success:
        print("Assertion circuit error")
    #invert the phase so the output state is |0000>
    circuit.u1(-pi/2, q[3])
    circuit.h(q[3])
    
    circuit.measure(q[0],c[0])
    circuit.measure(q[3],c[1])
    circuit.measure(q[2],c[2])
    circuit.measure(q[1],c[3])

        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QFT with superposition assertion, size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())


In [20]:
    qftExperimentClassical()
    qftExperimentSuperposition()

QFT without classical assertion: size = 50 depth =  34 width =  24 
 OPs = OrderedDict([('cx', 22), ('u1', 10), ('u2', 9), ('u3', 5), ('measure', 4), ('barrier', 1)]) subcircuits = 17
QFT with classical assertion: size = 59 depth =  41 width =  25 
 OPs = OrderedDict([('cx', 27), ('u3', 14), ('u1', 8), ('u2', 5), ('measure', 5), ('barrier', 1)]) subcircuits = 16
QFT without superposition assertion size = 49 depth =  36 width =  24 
 OPs = OrderedDict([('cx', 24), ('u1', 14), ('u2', 6), ('measure', 4), ('u3', 1), ('barrier', 1)]) subcircuits = 17
QFT with superposition assertion, size = 53 depth =  35 width =  25 
 OPs = OrderedDict([('cx', 25), ('u1', 13), ('u2', 7), ('measure', 5), ('u3', 3), ('barrier', 1)]) subcircuits = 16


In [21]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of QFT without classical assertion:

In [22]:
calcSuccessrate(res.get_counts(0), ["0000"], 0)

total_count =  8192 success_count =  1869 success_rate =  22.81494140625 %


Success rate of QFT with classical assertion:

In [24]:
calcSuccessrate(res.get_counts(1), ["0 0000"], 1)

total_count =  4286 success_count =  513 success_rate =  11.969202053196454 %


Success rate of QFT without superposition assertion:

In [25]:
calcSuccessrate(res.get_counts(2), ["0000"], 0)

total_count =  8192 success_count =  1845 success_rate =  22.52197265625 %


Success rate of QFT with superposition assertion

In [26]:
calcSuccessrate(res.get_counts(3), ["0 0000"], 1)

total_count =  3429 success_count =  810 success_rate =  23.62204724409449 %
